HOMEWORK 3

I consulted with Emma for this hw! 

From the DHS synthetic dataset, prepare an analysis file for mining association rules about services recipients use, run association analysis using mlxtend package

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import dhs_util
from dhs_util import *


os.chdir('/Users/katielark/Downloads/data_viz')
df = pd.read_csv('dhs_service_records_synthesized_final.csv')

df = dhs_preprocessing(df)
df, service_map = add_service_label(df)
df = add_age_bin(df)

recipient = get_recipient_attribute(df)

In [2]:
import mlxtend

In [3]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.preprocessing import *
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpmax
from mlxtend.frequent_patterns import hmine

In [4]:
serv_list = []
for groups in df.groupby('id').groups.values():
    serv_list.append(df.loc[groups]['serv'].tolist())

# following the tutorial example 
def oneHotCoding(serv_list):
    te = TransactionEncoder()
    te_ary = te.fit(serv_list).transform(serv_list)
    te_df = pd.DataFrame(te_ary, columns=te.columns_)
    return te_df

serv_oneHot = oneHotCoding(serv_list)

In [5]:
def get_id_service_matrix(df):
    df_temp = df.groupby(["id","serv"]).agg(
        num_serv = ('service', 'nunique')
    ).reset_index()
    df_serv = df_temp.pivot_table(
        values='num_serv', index=["id"],
        columns="serv", aggfunc=np.sum
    ).reset_index()
    return df_serv

In [6]:
df_id_serv = get_id_service_matrix(df) # this gives number of times the service is used
df_id_serv.iloc[:,1:23] = df_id_serv.iloc[:,1:23] > 0  # this converts value into True or False

/var/folders/00/yzv7lrx12xq5_g14dw7m_2180000gn/T/ipykernel_63700/4056837485.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_temp = df.groupby(["id","serv"]).agg(
/var/folders/00/yzv7lrx12xq5_g14dw7m_2180000gn/T/ipykernel_63700/4056837485.py:5: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  df_serv = df_temp.pivot_table(
/var/folders/00/yzv7lrx12xq5_g14dw7m_2180000gn/T/ipykernel_63700/4056837485.py:5: FutureWarning: The provided callable <function sum at 0x11254c860> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" ins

Frequent itemsets

1. Extract service itemsets that have support greater than 0.01using apriori
2. Extract service itemsets that have support greater than 0.0001 using fpgrowth
3. Extract service itemsets that have support greater than 0.0001 using hmine

Make observations of the difference between them.

In [7]:
#apriori 
apriori(df_id_serv.iloc[:,1:23], use_colnames=True, min_support=0.01)\
    .sort_values(by="support", ascending=False)

,support,itemsets
4,0.941422,(S12)
2,0.153844,(S09)
14,0.139131,"(S09, S12)"
15,0.103528,"(S09, S14)"
6,0.103528,(S14)
25,0.094436,"(S09, S12, S14)"
20,0.094436,"(S12, S14)"
3,0.040882,(S11)
18,0.032106,"(S11, S12)"
10,0.031396,(S19)


In [8]:
#fpgrowth
fpgrowth(df_id_serv.iloc[:,1:23], use_colnames=True, min_support=0.0001)\
    .sort_values(by="support", ascending=False)

,support,itemsets
0,0.941422,(S12)
1,0.153844,(S09)
22,0.139131,"(S09, S12)"
3,0.103528,(S14)
30,0.103528,"(S09, S14)"
...,...,...
218,0.000101,"(S18, S09, S16, S12, S21)"
505,0.000101,"(S09, S11, S14, S12, S06, S15)"
504,0.000101,"(S11, S14, S12, S06, S15, S21)"
219,0.000101,"(S18, S16, S14, S12, S21)"


In [9]:
#hmine
hmine(df_id_serv.iloc[:,1:23], use_colnames=True, min_support=0.0001)\
    .sort_values(by="support", ascending=False)

,support,itemsets
784,0.941422,(S12)
458,0.153844,(S09)
548,0.139131,"(S09, S12)"
633,0.103528,"(S09, S14)"
872,0.103528,(S14)
...,...,...
479,0.000101,"(S09, S10, S17)"
379,0.000101,"(S07, S10)"
380,0.000101,"(S07, S11, S10)"
715,0.000101,"(S19, S10)"


Observations: Apriori produces only 30 rows while fpgrowth and hmine produce 935 rows. Fpgrowth and hmine produce identical looking datasets, except for the indices. The support values are all very close to the lower bound listed in the code. 

4. Extract service itemsets that have support greater than 0.0001 using fpmax

Make observations between fpmax and others

In [10]:
#fpmax
fpmax(df_id_serv.iloc[:,1:23], use_colnames=True, min_support=0.0001)\
    .sort_values(by="support", ascending=False)

,support,itemsets
56,0.000326,"(S18, S09, S17, S14, S12, S15)"
1,0.000305,"(S01, S12)"
34,0.000253,"(S18, S09, S16, S11, S14, S12, S07)"
63,0.000247,"(S09, S11, S14, S12, S19)"
11,0.000223,"(S18, S09, S02, S14, S12, S03)"
...,...,...
16,0.000101,"(S07, S11, S10)"
15,0.000101,"(S19, S10)"
29,0.000101,"(S09, S11, S14, S12, S06, S15, S21)"
17,0.000101,"(S09, S17, S10)"


Observations: Fpmax produces 64 rows. The support values are much smaller than those in the other tables.

Frequent itemsets with length (size of the itemsets)

1. Extract length-1 service itemsets that have support greater than 0.01 using apriori
2. Extract length-2 service itemsets that have support greater than 0.005 using hmine
3. Extract length-5 service itemsets that have support greater than 0.0001 using fpmax

In [11]:
#length-1 apriori
apriori_01 = apriori(serv_oneHot,min_support=0.01,use_colnames=True)
apriori_01.iloc[0:12, :]

,support,itemsets
0,0.013687,(S03)
1,0.012649,(S05)
2,0.153844,(S09)
3,0.040882,(S11)
4,0.941422,(S12)
5,0.011915,(S13)
6,0.103528,(S14)
7,0.024002,(S15)
8,0.012967,(S17)
9,0.013573,(S18)


In [12]:
#length-2 hmine
hmine_005 = hmine(serv_oneHot,min_support=0.005,use_colnames=True)
hmine_005_2 = hmine_005[hmine_005['itemsets'].apply(lambda x: len(x)==2)]
hmine_005_2

,support,itemsets
1,0.005274,"(S09, S03)"
3,0.012284,"(S03, S12)"
5,0.008057,"(S05, S12)"
8,0.006398,"(S07, S11)"
10,0.005105,"(S07, S12)"
12,0.018307,"(S09, S11)"
16,0.139131,"(S09, S12)"
27,0.008419,"(S13, S09)"
28,0.103528,"(S09, S14)"
32,0.019431,"(S09, S15)"


In [13]:
#length-5 fpmax
fpmax_0001 = fpmax(serv_oneHot,min_support=0.0001,use_colnames=True)
fpmax_0001_2 = fpmax_0001[fpmax_0001['itemsets'].apply(lambda x: len(x)==5)]
fpmax_0001_2

,support,itemsets
4,0.000131,"(S09, S04, S14, S12, S03)"
18,0.000163,"(S18, S09, S10, S14, S12)"
19,0.000157,"(S09, S10, S12, S06, S21)"
21,0.000124,"(S09, S10, S12, S15, S21)"
22,0.000208,"(S09, S10, S14, S12, S15)"
23,0.000139,"(S09, S10, S14, S12, S21)"
24,0.000197,"(S09, S11, S10, S14, S12)"
25,0.000103,"(S09, S16, S14, S06, S21)"
31,0.000103,"(S09, S11, S12, S07, S19)"
33,0.000120,"(S09, S11, S12, S07, S03)"


Association rules

1. Extract association rules that have lift greater than 0.5, using itemsets from fpgrowth with support greater than 0.05
2. Extract assocaition rules that have support greater than 0.0003, using itemsets from fpmax with support greater than 0.0001

In [14]:
def serv_rules(freq_itemsets,metrics,threshold):
    asso_rules = association_rules(freq_itemsets, metric=metrics, min_threshold=threshold)
    return asso_rules.sort_values(by='lift', ascending=False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

In [15]:
min_freq = 1000 # if we want to set threshold by frequency of the itemsets
min_support = min_freq/serv_oneHot.shape[0]
min_confidence = 0.6
min_rule_support = 0.2
min_lift = 0.15

In [16]:
#association rules w fpgrowth
freq_itemset_fpgrowth = fpgrowth(serv_oneHot,min_support=min_support,use_colnames=True)
freq_itemset_fpgrowth.describe()

,support
count,146.000000
mean,0.017443
std,0.080320
min,0.001875
25%,0.002868
50%,0.004436
75%,0.009374
max,0.941422


In [17]:
rule_fpgrowth = serv_rules(freq_itemset_fpgrowth,"support",0.05)
rule_fpgrowth

,antecedents,consequents,support,confidence,lift
6,"(S09, S12)",(S14),0.094436,0.678758,6.556292
11,(S14),"(S09, S12)",0.094436,0.912184,6.556292
2,(S09),(S14),0.103528,0.672938,6.500073
3,(S14),(S09),0.103528,1.000000,6.500073
8,"(S12, S14)",(S09),0.094436,1.000000,6.500073
9,(S09),"(S12, S14)",0.094436,0.613843,6.500073
4,(S12),(S14),0.094436,0.100312,0.968942
5,(S14),(S12),0.094436,0.912184,0.968942
7,"(S09, S14)",(S12),0.094436,0.912184,0.968942
10,(S12),"(S09, S14)",0.094436,0.100312,0.968942


In [18]:
#association rules with fpmax
freq_itemset_fpmax = fpmax(serv_oneHot,min_support=0.0001, use_colnames=True)
freq_itemset_fpmax.describe()

,support
count,64.000000
mean,0.000148
std,0.000050
min,0.000101
25%,0.000114
50%,0.000129
75%,0.000164
max,0.000326


In [19]:
asso_rules = association_rules(freq_itemset_fpmax, metric='lift', min_threshold=0.0003, support_only=True)
asso_rules.sort_values(by='support', ascending=False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

,antecedents,consequents,support,confidence,lift
32,"(S18, S12, S15)","(S09, S17, S14)",0.000326,NaN,NaN
48,"(S09, S17)","(S18, S14, S12, S15)",0.000326,NaN,NaN
35,"(S09, S17, S15)","(S18, S14, S12)",0.000326,NaN,NaN
36,"(S09, S14, S12)","(S18, S17, S15)",0.000326,NaN,NaN
37,"(S09, S14, S15)","(S18, S17, S12)",0.000326,NaN,NaN
...,...,...,...,...,...
29,"(S18, S17, S15)","(S09, S14, S12)",0.000326,NaN,NaN
30,"(S18, S14, S12)","(S09, S17, S15)",0.000326,NaN,NaN
3,"(S18, S09, S17, S14, S15)",(S12),0.000326,NaN,NaN
1,(S12),(S01),0.000305,NaN,NaN


Make predictions

If you were to make suggestions for DHS to consider increasing the offering of one service, say, "Families_Receiving_Child_Welfare_Services" (S06), what other services would you suggest to offer together with? 
1. Run predict({"S06"}, rules, consequents_only=False), with rules generated from different thresholds and algorithms

In [20]:
def predict(antecedent, rules, consequents_only = False):
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    if consequents_only:
        # a way to convert a frozen set with one element to string
        preds = preds['consequents'].apply(iter).apply(next)
    return preds

In [25]:
asso_rules = association_rules(freq_itemset_fpmax, metric='lift', min_threshold=0.000001, support_only=True)
asso_rules.sort_values(by='lift', ascending=False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

,antecedents,consequents,support,confidence,lift
0,(S08),(S12),0.000199,NaN,NaN
1,(S12),(S08),0.000199,NaN,NaN
2,(S01),(S12),0.000305,NaN,NaN
3,(S12),(S01),0.000305,NaN,NaN
4,(S22),(S12),0.000124,NaN,NaN
...,...,...,...,...,...
2455,(S09),"(S19, S11, S14, S12)",0.000247,NaN,NaN
2456,(S11),"(S19, S09, S14, S12)",0.000247,NaN,NaN
2457,(S14),"(S19, S09, S11, S12)",0.000247,NaN,NaN
2458,(S12),"(S19, S09, S11, S14)",0.000247,NaN,NaN


In [26]:
predict({"S06"}, asso_rules, consequents_only=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
402,(S06),"(S09, S10, S12, S21)",NaN,NaN,0.000157,NaN,NaN,NaN,NaN,NaN
566,(S06),"(S09, S16, S14, S21)",NaN,NaN,0.000103,NaN,NaN,NaN,NaN,NaN
628,(S06),"(S18, S09, S14, S12, S21)",NaN,NaN,0.000146,NaN,NaN,NaN,NaN,NaN
690,(S06),"(S09, S17, S14, S12, S21)",NaN,NaN,0.000105,NaN,NaN,NaN,NaN,NaN
751,(S06),"(S09, S17, S12, S15, S21)",NaN,NaN,0.000114,NaN,NaN,NaN,NaN,NaN
877,(S06),"(S09, S11, S14, S12, S15, S21)",NaN,NaN,0.000101,NaN,NaN,NaN,NaN,NaN


In [27]:
asso_rules = association_rules(freq_itemset_fpgrowth, metric='lift', min_threshold=0.000001, support_only=True)
asso_rules.sort_values(by='lift', ascending=False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

,antecedents,consequents,support,confidence,lift
0,(S09),(S12),0.139131,NaN,NaN
1,(S12),(S09),0.139131,NaN,NaN
2,(S11),(S12),0.032106,NaN,NaN
3,(S12),(S11),0.032106,NaN,NaN
4,(S09),(S11),0.018307,NaN,NaN
...,...,...,...,...,...
779,"(S09, S14)",(S02),0.001875,NaN,NaN
780,"(S02, S14)",(S09),0.001875,NaN,NaN
781,(S09),"(S02, S14)",0.001875,NaN,NaN
782,(S02),"(S09, S14)",0.001875,NaN,NaN


In [28]:
predict({"S06"}, asso_rules, consequents_only=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
464,(S06),(S21),NaN,NaN,0.004582,NaN,NaN,NaN,NaN,NaN
467,(S06),(S12),NaN,NaN,0.003439,NaN,NaN,NaN,NaN,NaN
469,(S06),(S09),NaN,NaN,0.002898,NaN,NaN,NaN,NaN,NaN
474,(S06),"(S12, S21)",NaN,NaN,0.003215,NaN,NaN,NaN,NaN,NaN
480,(S06),"(S09, S21)",NaN,NaN,0.002804,NaN,NaN,NaN,NaN,NaN
487,(S06),"(S09, S12)",NaN,NaN,0.002518,NaN,NaN,NaN,NaN,NaN
500,(S06),"(S09, S12, S21)",NaN,NaN,0.002486,NaN,NaN,NaN,NaN,NaN


If I were to make suggestions for DHS to consider increasing the offering of S06, I would suggest also increasing S09, S12, S21 and S14. 

In [29]:
predict({"S15"}, asso_rules, consequents_only=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
107,(S15),(S12),NaN,NaN,0.022561,NaN,NaN,NaN,NaN,NaN
109,(S15),(S09),NaN,NaN,0.019431,NaN,NaN,NaN,NaN,NaN
111,(S15),(S14),NaN,NaN,0.009888,NaN,NaN,NaN,NaN,NaN
113,(S15),(S11),NaN,NaN,0.003449,NaN,NaN,NaN,NaN,NaN
119,(S15),"(S09, S12)",NaN,NaN,0.018280,NaN,NaN,NaN,NaN,NaN
125,(S15),"(S09, S14)",NaN,NaN,0.009888,NaN,NaN,NaN,NaN,NaN
129,(S15),"(S12, S14)",NaN,NaN,0.009342,NaN,NaN,NaN,NaN,NaN
144,(S15),"(S09, S12, S14)",NaN,NaN,0.009342,NaN,NaN,NaN,NaN,NaN
151,(S15),"(S09, S11)",NaN,NaN,0.003335,NaN,NaN,NaN,NaN,NaN
157,(S15),"(S11, S12)",NaN,NaN,0.003209,NaN,NaN,NaN,NaN,NaN


If I were to make suggestions for DHS to consider increasing the offering of S15, I would suggest also increasing S14, S09, S12 and S21.